In [1]:
from azureml.core import Workspace, Experiment

# ws = Workspace.get(name="udacity-project")
ws = Workspace.from_config()
ws.write_config(path='.azureml')
#exp = Experiment(workspace=ws, name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-hyperdrive")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-150229
Azure region: southcentralus
Subscription id: 9e65f93e-bdd8-437b-b1e8-0647cd6098f7
Resource group: aml-quickstarts-150229


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
aml_compute_cluster_name = "cpu-cluster"

# Verify that cluster doesn't exist already
try:
    aml_compute = ComputeTarget(workspace=ws, name=aml_compute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2",
                                                           min_nodes=1,
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(workspace=ws, 
                                       name=aml_compute_cluster_name, 
                                       provisioning_configuration=compute_config)

aml_compute.wait_for_completion(show_output=True)

Creating......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded........................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
compute_targets = ws.compute_targets

for i, key in enumerate(compute_targets):
    print(f"{i+1}. Compute target\n\tname: {compute_targets[key].name}\n\tType: {compute_targets[key].type}")


1. Compute target
	name: compute-instance-rvl
	Type: ComputeInstance
2. Compute target
	name: cpu-cluster
	Type: AmlCompute


In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
import shutil

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        "--C": uniform(0.001, 1.0),
        "--max_iter": choice(5, 25, 50, 100, 150)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

script_folder = "./training"

if "training" not in os.listdir():
    os.mkdir(script_folder)
    
shutil.copy('./train.py', script_folder)
    
# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory=script_folder,
              compute_target=aml_compute,
              entry_script='train.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                     hyperparameter_sampling=ps,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=50,
                                     max_concurrent_runs=4,
                                     policy=policy)


'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hd_experiment_run_submission = exp.submit(config=hyperdrive_config)
RunDetails(hd_experiment_run_submission).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [21]:
import joblib
# Get your best run and save the model from that run.

best_run = hd_experiment_run_submission.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Experiment Run:')
print(f" Best Run Id: {best_run.id}")
print(f" Accuracy: {best_run_metrics['Accuracy']}")
print(f" Regularization Strength: {best_run_metrics['Regularization Strength:']}")
print(f" Max iterations: {best_run_metrics['Max iterations:']}")

# Save the best model 
final_model_hd = best_run.register_model(model_name='hyperdrive_model', model_path='./outputs/model.joblib')
final_model_hd.download(target_dir='outputs', exist_ok=True)

Best Experiment Run:
 Best Run Id: HD_fb610ace-5b83-40f8-b09d-41ae9574a8ac_0
 Accuracy: 0.9157562515173586
 Regularization Strength: 0.72348698662538
 Max iterations: 100


'outputs/model.joblib'

In [24]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

url_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=url_path)

In [37]:
from train import clean_data
import pandas as pd
from sklearn.model_selection import train_test_split
from azureml.core import Dataset

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

# Concatenate features with label column
all_data_df = pd.concat([x, y], axis=1)

# split data into training and testing data
train_data, test_data = train_test_split(all_data_df, test_size=0.25)

# Save training data to csv file
train_data.to_csv('./training/train_data.csv')

# Read saved training data and create a dataset in Azure ML
data_store = ws.get_default_datastore()
data_store.upload(src_dir='training', target_path='training_data')
train_ds = TabularDatasetFactory.from_delimited_files(path=[(data_store, 'training_data/train_data.csv')])

Uploading an estimated of 4 files
Target already exists. Skipping upload for training_data/.amlignore
Target already exists. Skipping upload for training_data/.amlignore.amltmp
Target already exists. Skipping upload for training_data/train.py
Target already exists. Skipping upload for training_data/train_data.csv
Uploaded 0 files


In [42]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_ds,
    label_column_name='y',
    n_cross_validations=7,
    compute_target=aml_compute,
    experiment_exit_score=0.98,
    enable_early_stopping=True)

In [43]:
# Create new experiment
exp_automl = Experiment(workspace=ws, name="udacity-automl")

# Submit your automl run
automl_experiment_run_submission = exp.submit(config=automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_5dbc0476-f0fa-46a4-8d19-0e3dc30fc1f9,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the sm

In [53]:
# Retrieve and save your best automl model.

best_run, automl_model = automl_experiment_run_submission.get_output()

joblib.dump(automl_model, filename='./outputs/automl_model.joblib')
best_run.register_model(model_name='automl_model', model_path='./outputs/')


Model(workspace=Workspace.create(name='quick-starts-ws-150229', subscription_id='9e65f93e-bdd8-437b-b1e8-0647cd6098f7', resource_group='aml-quickstarts-150229'), name=automl_model.pkl, id=automl_model.pkl:1, version=1, tags={}, properties={})

Cleanup: Remove the cluster after training

In [55]:
aml_compute.delete()